# CSc 180 Project 2

## Alec Resha

# Import Statements and Def's

In [1]:
#@title Import Statements

import pandas as pd
import sklearn as sk
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
from collections.abc import Sequence
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import time

In [2]:
#@title Helpful functions
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

# Data PreProcessing

In [3]:
df = pd.read_csv("network_intrusion_data.csv",na_values=['NA', '?'])

In [4]:
#@title Basic Preprocessing

# Setting column titles
df.columns = [
              'duration',
              'protocol_type',
              'service',
              'flag',
              'src_bytes',
              'dst_bytes',
              'land',
              'wrong_fragment',
              'urgent',
              'hot',
              'num_failed_logins',
              'logged_in',
              'num_compromised',
              'root_shell',
              'su_attempted',
              'num_root',
              'num_file_creations',
              'num_shells',
              'num_access_files',
              'num_outbound_cmds',
              'is_host_login',
              'is_guest_login',
              'count',
              'srv_count',
              'serror_rate',
              'srv_serror_rate',
              'rerror_rate',
              'srv_rerror_rate',
              'same_srv_rate',
              'diff_srv_rate',
              'srv_diff_host_rate',
              'dst_host_count',
              'dst_host_srv_count',
              'dst_host_same_srv_rate',
              'dst_host_diff_srv_rate',
              'dst_host_same_src_port_rate',
              'dst_host_srv_diff_host_rate',
              'dst_host_serror_rate',
              'dst_host_srv_serror_rate',
              'dst_host_rerror_rate',
              'dst_host_srv_rerror_rate',
              'outcome'
              ]

# Remove redundant records
df.drop_duplicates(inplace=True)

# Drop any rows that are missing values
df.dropna(axis='index', how='any',inplace=True)

In [5]:
#@title Encoding and Normalizing data

# Normalize number columns, drop NaN columns
df_num = df.select_dtypes(include=[np.number])
df_norm = (df_num - df_num.mean()) / (df_num.max() - df_num.min())
df[df_norm.columns] = df_norm
df.dropna(axis='columns',how='all',inplace=True)

# Encode all string columns
attacks = encode_text_index(df,'outcome')
protocol_types = encode_text_index(df,'protocol_type')
services = encode_text_index(df,'service')
flags = encode_text_index(df,'flag')

# Replace outcome with 0 or 1. 11 is index of 'normal.'
df["outcome"] = (df[['outcome']]==11).astype(int)

In [6]:
#@title Split up data

x,y = to_xy(df,'outcome')
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=32)

In [7]:
# For CNN
x2 = x.reshape(x.shape[0], 1, x.shape[1], 1)
y2 = y
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2,y2,test_size=0.25, random_state=32)

# Fully Connected Neural Networks

## Fully Connected Network 1

### Running Model

In [8]:
#@title Create and Train Model
model = Sequential()

model.add(Dense(50, input_dim = x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath="fcn/best_weights1.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=100)

Epoch 1/100
3413/3413 - 3s - loss: 0.0704 - val_loss: 0.0546
Epoch 2/100
3413/3413 - 2s - loss: 0.0362 - val_loss: 0.0316
Epoch 3/100
3413/3413 - 2s - loss: 0.0289 - val_loss: 0.0373
Epoch 4/100
3413/3413 - 2s - loss: 0.0224 - val_loss: 0.0164
Epoch 5/100
3413/3413 - 2s - loss: 0.0169 - val_loss: 0.0342
Epoch 6/100
3413/3413 - 2s - loss: 0.0144 - val_loss: 0.0216
Epoch 00006: early stopping


### Predict Model

In [9]:
#@title Load best weights
model.load_weights('fcn/best_weights1.hdf5') # load weights from best model

In [10]:
#@title Predict and RMSE Score
pred=model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final RMSE score: {:.4}".format(score))

Final RMSE score: 0.06235


In [11]:
y_true = np.argmax(y_test,axis=1)
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

Accuracy: 0.9956040332994478
Averaged F1: 0.9956058050707804
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     14406
           1       1.00      0.99      1.00     21991

    accuracy                           1.00     36397
   macro avg       1.00      1.00      1.00     36397
weighted avg       1.00      1.00      1.00     36397



### Confusion Matrix

In [12]:
pred=model.predict(x_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=pred).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.51 0.49]
 [1.   0.  ]]


## Fully Connected Network 2

### Running Model

In [13]:
#@title Create and Train Model
model = Sequential()

model.add(Dense(25, input_dim = x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath="fcn/best_weights2.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=100)

Epoch 1/100
3413/3413 - 2s - loss: 0.0830 - val_loss: 0.0491
Epoch 2/100
3413/3413 - 2s - loss: 0.0372 - val_loss: 0.0260
Epoch 3/100
3413/3413 - 2s - loss: 0.0301 - val_loss: 0.0264
Epoch 4/100
3413/3413 - 2s - loss: 0.0268 - val_loss: 0.0275
Epoch 00004: early stopping


### Predict Model

In [14]:
#@title Load best weights
model.load_weights('fcn/best_weights2.hdf5') # load weights from best model

In [15]:
#@title Predict and RMSE Score
pred=model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final RMSE score: {:.4}".format(score))

Final RMSE score: 0.08135


In [16]:
y_true = np.argmax(y_test,axis=1)
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

Accuracy: 0.9913454405582878
Averaged F1: 0.9913382034304721
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     14406
           1       0.99      1.00      0.99     21991

    accuracy                           0.99     36397
   macro avg       0.99      0.99      0.99     36397
weighted avg       0.99      0.99      0.99     36397



### Confusion Matrix

In [17]:
pred=model.predict(x_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=pred).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.52 0.48]
 [1.   0.  ]]


## Fully Connected Network 3

### Running Model

In [18]:
#@title Create and Train Model
model = Sequential()

model.add(Dense(50, input_dim = x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath="fcn/best_weights3.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=100)

Epoch 1/100
3413/3413 - 3s - loss: 0.0598 - val_loss: 0.0329
Epoch 2/100
3413/3413 - 2s - loss: 0.0314 - val_loss: 0.0211
Epoch 3/100
3413/3413 - 2s - loss: 0.0240 - val_loss: 0.0203
Epoch 4/100
3413/3413 - 2s - loss: 0.0177 - val_loss: 0.0124
Epoch 5/100
3413/3413 - 2s - loss: 0.0160 - val_loss: 0.0199
Epoch 6/100
3413/3413 - 2s - loss: 0.0137 - val_loss: 0.0127
Epoch 00006: early stopping


### Predict Model

In [19]:
#@title Load best weights
model.load_weights('fcn/best_weights3.hdf5') # load weights from best model

In [20]:
#@title Predict and RMSE Score
pred=model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final RMSE score: {:.4}".format(score))

Final RMSE score: 0.05447


In [21]:
y_true = np.argmax(y_test,axis=1)
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

Accuracy: 0.9964282770558013
Averaged F1: 0.9964284905372138
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14406
           1       1.00      1.00      1.00     21991

    accuracy                           1.00     36397
   macro avg       1.00      1.00      1.00     36397
weighted avg       1.00      1.00      1.00     36397



### Confusion Matrix

In [22]:
pred=model.predict(x_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=pred).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.55 0.45]
 [1.   0.  ]]


## Fully Connected Network 4

### Running Model

In [23]:
#@title Create and Train Model
model = Sequential()

model.add(Dense(50, input_dim = x_train.shape[1], activation='sigmoid'))
model.add(Dense(25, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath="fcn/best_weights4.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=100)

Epoch 1/100
3413/3413 - 2s - loss: 0.0941 - val_loss: 0.0514
Epoch 2/100
3413/3413 - 2s - loss: 0.0432 - val_loss: 0.0371
Epoch 3/100
3413/3413 - 2s - loss: 0.0343 - val_loss: 0.0261
Epoch 4/100
3413/3413 - 2s - loss: 0.0293 - val_loss: 0.0265
Epoch 5/100
3413/3413 - 2s - loss: 0.0250 - val_loss: 0.0230
Epoch 6/100
3413/3413 - 2s - loss: 0.0218 - val_loss: 0.0176
Epoch 7/100
3413/3413 - 2s - loss: 0.0188 - val_loss: 0.0216
Epoch 8/100
3413/3413 - 2s - loss: 0.0166 - val_loss: 0.0143
Epoch 9/100
3413/3413 - 2s - loss: 0.0152 - val_loss: 0.0115
Epoch 10/100
3413/3413 - 2s - loss: 0.0134 - val_loss: 0.0106
Epoch 11/100
3413/3413 - 2s - loss: 0.0129 - val_loss: 0.0201
Epoch 00011: early stopping


### Predict Model

In [24]:
#@title Load best weights
model.load_weights('fcn/best_weights4.hdf5') # load weights from best model

In [25]:
#@title Predict and RMSE Score
pred=model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final RMSE score: {:.4}".format(score))

Final RMSE score: 0.0496


In [26]:
y_true = np.argmax(y_test,axis=1)
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

Accuracy: 0.996840398933978
Averaged F1: 0.9968406820034982
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14406
           1       1.00      1.00      1.00     21991

    accuracy                           1.00     36397
   macro avg       1.00      1.00      1.00     36397
weighted avg       1.00      1.00      1.00     36397



### Confusion Matrix

In [27]:
pred=model.predict(x_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=pred).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.62 0.38]
 [1.   0.  ]]


## Fully Connected Network 5

### Running Model

In [28]:
#@title Create and Train Model
model = Sequential()

model.add(Dense(50, input_dim = x_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath="fcn/best_weights5.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=100)

Epoch 1/100
3413/3413 - 2s - loss: 0.1509 - val_loss: 0.0902
Epoch 2/100
3413/3413 - 2s - loss: 0.0994 - val_loss: 0.0764
Epoch 3/100
3413/3413 - 2s - loss: 0.0840 - val_loss: 0.0636
Epoch 4/100
3413/3413 - 2s - loss: 0.0756 - val_loss: 0.0650
Epoch 5/100
3413/3413 - 2s - loss: 0.0681 - val_loss: 0.0527
Epoch 6/100
3413/3413 - 2s - loss: 0.0651 - val_loss: 0.0596
Epoch 7/100
3413/3413 - 2s - loss: 0.0658 - val_loss: 0.0478
Epoch 8/100
3413/3413 - 2s - loss: 0.0596 - val_loss: 2.2373
Epoch 9/100
3413/3413 - 2s - loss: 0.0593 - val_loss: 0.0574
Epoch 00009: early stopping


### Predict Model

In [29]:
#@title Load best weights
model.load_weights('fcn/best_weights5.hdf5') # load weights from best model

In [30]:
#@title Predict and RMSE Score
pred=model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final RMSE score: {:.4}".format(score))

Final RMSE score: 0.1094


In [31]:
y_true = np.argmax(y_test,axis=1)
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

Accuracy: 0.9852460367612715
Averaged F1: 0.9852146309451715
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     14406
           1       0.98      1.00      0.99     21991

    accuracy                           0.99     36397
   macro avg       0.99      0.98      0.98     36397
weighted avg       0.99      0.99      0.99     36397



### Confusion Matrix

In [32]:
pred=model.predict(x_test)
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=pred).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.62 0.38]
 [1.   0.  ]]


# Convolutional Neural Network

## CNN 1

### Model Definition

In [33]:
start_time=time.time()

cnn = Sequential()
cnn.add(Conv2D(
    filters=64,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu',
    input_shape=(1, x.shape[1], 1)))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=128,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Flatten())
cnn.add(Dense(1028, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath='cnn/best_weights1.hdf5', verbose=0, save_weights_only=True, save_best_only=True) # save best model

cnn.fit(
    x_train2,
    y_train2,
    batch_size=128,
    epochs=2,
    verbose=2,
    validation_data=(x_test2, y_test2),
    callbacks=[monitor,checkpointer]
)

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Epoch 1/2
854/854 - 13s - loss: 0.0518 - accuracy: 0.9826 - val_loss: 0.0164 - val_accuracy: 0.9956
Epoch 2/2
854/854 - 13s - loss: 0.0161 - accuracy: 0.9953 - val_loss: 0.0118 - val_accuracy: 0.9966
Elapsed time: 0:00:25.72


### Predict

In [34]:
#@title Load best weights
cnn.load_weights('cnn/best_weights1.hdf5')

In [35]:
y_true2 = np.argmax(y_test2,axis=1)
pred2 = cnn.predict(x_test2)
pred2 = np.argmax(pred2,axis=1)

score = np.sqrt(metrics.mean_squared_error(pred2,y_true2))
print("Final RMSE score: {:.4}".format(score))

score = metrics.accuracy_score(y_true2, pred2)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true2, pred2, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true2, pred2))

Final RMSE score: 0.0579
Accuracy: 0.9966480753908289
Averaged F1: 0.996648115505803
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14406
           1       1.00      1.00      1.00     21991

    accuracy                           1.00     36397
   macro avg       1.00      1.00      1.00     36397
weighted avg       1.00      1.00      1.00     36397



### Confusion Matrix

In [36]:
pred2=cnn.predict(x_test2)
con_mat = tf.math.confusion_matrix(labels=y_test2, predictions=pred2).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.62 0.38]
 [1.   0.  ]]


## CNN 2

### Model Definition

In [37]:
start_time=time.time()

cnn = Sequential()
cnn.add(Conv2D(
    filters=16,
    kernel_size=(1,y.shape[1]),
    strides=(3,1),
    activation='relu',
    input_shape=(1, x.shape[1], 1)))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=32,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Flatten())
cnn.add(Dense(1028, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath='cnn/best_weights2.hdf5', verbose=0, save_weights_only=True, save_best_only=True) # save best model

cnn.fit(
    x_train2,
    y_train2,
    batch_size=128,
    epochs=5,
    verbose=2,
    validation_data=(x_test2, y_test2),
    callbacks=[monitor,checkpointer]
)

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Epoch 1/5
854/854 - 5s - loss: 0.0676 - accuracy: 0.9778 - val_loss: 0.0276 - val_accuracy: 0.9923
Epoch 2/5
854/854 - 5s - loss: 0.0227 - accuracy: 0.9927 - val_loss: 0.0130 - val_accuracy: 0.9966
Epoch 3/5
854/854 - 5s - loss: 0.0150 - accuracy: 0.9960 - val_loss: 0.0107 - val_accuracy: 0.9969
Epoch 4/5
854/854 - 5s - loss: 0.0130 - accuracy: 0.9961 - val_loss: 0.0200 - val_accuracy: 0.9925
Epoch 5/5
854/854 - 5s - loss: 0.0122 - accuracy: 0.9964 - val_loss: 0.0098 - val_accuracy: 0.9969
Epoch 00005: early stopping
Elapsed time: 0:00:25.31


### Predict

In [38]:
#@title Load best weights
cnn.load_weights('cnn/best_weights2.hdf5')

In [39]:
y_true2 = np.argmax(y_test2,axis=1)
pred2 = cnn.predict(x_test2)
pred2 = np.argmax(pred2,axis=1)

score = np.sqrt(metrics.mean_squared_error(pred2,y_true2))
print("Final RMSE score: {:.4}".format(score))

score = metrics.accuracy_score(y_true2, pred2)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true2, pred2, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true2, pred2))

Final RMSE score: 0.05572
Accuracy: 0.996895348517735
Averaged F1: 0.9968948822671937
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14406
           1       1.00      1.00      1.00     21991

    accuracy                           1.00     36397
   macro avg       1.00      1.00      1.00     36397
weighted avg       1.00      1.00      1.00     36397



### Confusion Matrix

In [40]:
pred2=cnn.predict(x_test2)
con_mat = tf.math.confusion_matrix(labels=y_test2, predictions=pred2).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.59 0.41]
 [1.   0.  ]]


## CNN 3

### Model Definition

In [41]:
start_time=time.time()

cnn = Sequential()
cnn.add(Conv2D(
    filters=64,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='sigmoid',
    input_shape=(1, x.shape[1], 1)))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=128,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='sigmoid'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Flatten())
cnn.add(Dense(1028, activation='sigmoid'))
cnn.add(Dropout(0.5))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath='cnn/best_weights3.hdf5', verbose=0, save_weights_only=True, save_best_only=True) # save best model

cnn.fit(
    x_train2,
    y_train2,
    batch_size=128,
    epochs=5,
    verbose=2,
    validation_data=(x_test2, y_test2),
    callbacks=[monitor,checkpointer]
)

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Epoch 1/5
854/854 - 13s - loss: 0.2689 - accuracy: 0.8881 - val_loss: 0.0853 - val_accuracy: 0.9752
Epoch 2/5
854/854 - 13s - loss: 0.0820 - accuracy: 0.9770 - val_loss: 0.0606 - val_accuracy: 0.9831
Epoch 3/5
854/854 - 13s - loss: 0.0695 - accuracy: 0.9818 - val_loss: 0.0587 - val_accuracy: 0.9846
Epoch 4/5
854/854 - 13s - loss: 0.0651 - accuracy: 0.9824 - val_loss: 0.0555 - val_accuracy: 0.9843
Epoch 5/5
854/854 - 13s - loss: 0.0610 - accuracy: 0.9829 - val_loss: 0.0497 - val_accuracy: 0.9838
Elapsed time: 0:01:04.41


### Predict

In [42]:
#@title Load best weights
cnn.load_weights('cnn/best_weights3.hdf5')

In [43]:
y_true2 = np.argmax(y_test2,axis=1)
pred2 = cnn.predict(x_test2)
pred2 = np.argmax(pred2,axis=1)

score = np.sqrt(metrics.mean_squared_error(pred2,y_true2))
print("Final RMSE score: {:.4}".format(score))

score = metrics.accuracy_score(y_true2, pred2)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true2, pred2, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true2, pred2))

Final RMSE score: 0.1272
Accuracy: 0.9838173475835921
Averaged F1: 0.9837850228009662
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     14406
           1       0.98      0.99      0.99     21991

    accuracy                           0.98     36397
   macro avg       0.99      0.98      0.98     36397
weighted avg       0.98      0.98      0.98     36397



### Confusion Matrix

In [44]:
pred2=cnn.predict(x_test2)
con_mat = tf.math.confusion_matrix(labels=y_test2, predictions=pred2).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.58 0.42]
 [1.   0.  ]]


## CNN 4

### Model Definition

In [45]:
start_time=time.time()

cnn = Sequential()
cnn.add(Conv2D(
    filters=64,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu',
    input_shape=(1, x.shape[1], 1)))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=32,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Flatten())
cnn.add(Dense(1028, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath='cnn/best_weights4.hdf5', verbose=0, save_weights_only=True, save_best_only=True) # save best model

cnn.fit(
    x_train2,
    y_train2,
    batch_size=128,
    epochs=5,
    verbose=2,
    validation_data=(x_test2, y_test2),
    callbacks=[monitor,checkpointer]
)

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Epoch 1/5
854/854 - 7s - loss: 0.2637 - accuracy: 0.8936 - val_loss: 0.1863 - val_accuracy: 0.9218
Epoch 2/5
854/854 - 7s - loss: 0.1625 - accuracy: 0.9411 - val_loss: 0.1421 - val_accuracy: 0.9324
Epoch 3/5
854/854 - 7s - loss: 0.1202 - accuracy: 0.9658 - val_loss: 0.1029 - val_accuracy: 0.9683
Epoch 4/5
854/854 - 7s - loss: 0.0991 - accuracy: 0.9702 - val_loss: 0.0854 - val_accuracy: 0.9709
Epoch 5/5
854/854 - 7s - loss: 0.0855 - accuracy: 0.9735 - val_loss: 0.0732 - val_accuracy: 0.9743
Elapsed time: 0:00:33.70


### Predict

In [46]:
#@title Load best weights
cnn.load_weights('cnn/best_weights4.hdf5')

In [47]:
y_true2 = np.argmax(y_test2,axis=1)
pred2 = cnn.predict(x_test2)
pred2 = np.argmax(pred2,axis=1)

score = np.sqrt(metrics.mean_squared_error(pred2,y_true2))
print("Final RMSE score: {:.4}".format(score))

score = metrics.accuracy_score(y_true2, pred2)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true2, pred2, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true2, pred2))

Final RMSE score: 0.1604
Accuracy: 0.9742835948017694
Averaged F1: 0.9741376672276821
              precision    recall  f1-score   support

           0       1.00      0.94      0.97     14406
           1       0.96      1.00      0.98     21991

    accuracy                           0.97     36397
   macro avg       0.98      0.97      0.97     36397
weighted avg       0.98      0.97      0.97     36397



### Confusion Matrix

In [48]:
pred2=cnn.predict(x_test2)
con_mat = tf.math.confusion_matrix(labels=y_test2, predictions=pred2).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.62 0.38]
 [1.   0.  ]]


## CNN 5

### Model Definition

In [49]:
start_time=time.time()

cnn = Sequential()
cnn.add(Conv2D(
    filters=8,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu',
    input_shape=(1, x.shape[1], 1)))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=8,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=8,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Conv2D(
    filters=8,
    kernel_size=(1,y.shape[1]),
    strides=(1,1),
    activation='relu'))
cnn.add(MaxPooling2D(pool_size=(1,2)))
cnn.add(Flatten())
cnn.add(Dense(1028, activation='relu'))
cnn.add(Dropout(0.25))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath='cnn/best_weights5.hdf5', verbose=0, save_weights_only=True, save_best_only=True) # save best model

cnn.fit(
    x_train2,
    y_train2,
    batch_size=128,
    epochs=5,
    verbose=2,
    validation_data=(x_test2, y_test2),
    callbacks=[monitor,checkpointer]
)

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Epoch 1/5
854/854 - 4s - loss: 0.1096 - accuracy: 0.9611 - val_loss: 0.0556 - val_accuracy: 0.9838
Epoch 2/5
854/854 - 3s - loss: 0.0542 - accuracy: 0.9834 - val_loss: 0.0592 - val_accuracy: 0.9790
Epoch 3/5
854/854 - 3s - loss: 0.0439 - accuracy: 0.9855 - val_loss: 0.0346 - val_accuracy: 0.9885
Epoch 4/5
854/854 - 3s - loss: 0.0294 - accuracy: 0.9899 - val_loss: 0.0218 - val_accuracy: 0.9934
Epoch 5/5
854/854 - 3s - loss: 0.0237 - accuracy: 0.9922 - val_loss: 0.0216 - val_accuracy: 0.9924
Elapsed time: 0:00:17.20


### Predict

In [50]:
#@title Load best weights
cnn.load_weights('cnn/best_weights5.hdf5')

In [51]:
y_true2 = np.argmax(y_test2,axis=1)
pred2 = cnn.predict(x_test2)
pred2 = np.argmax(pred2,axis=1)

score = np.sqrt(metrics.mean_squared_error(pred2,y_true2))
print("Final RMSE score: {:.4}".format(score))

score = metrics.accuracy_score(y_true2, pred2)
print('Accuracy: {}'.format(score))

f1 = metrics.f1_score(y_true2, pred2, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true2, pred2))

Final RMSE score: 0.08708
Accuracy: 0.9924169574415473
Averaged F1: 0.9924159557228742
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     14406
           1       0.99      0.99      0.99     21991

    accuracy                           0.99     36397
   macro avg       0.99      0.99      0.99     36397
weighted avg       0.99      0.99      0.99     36397



### Confusion Matrix

In [52]:
pred2=cnn.predict(x_test2)
con_mat = tf.math.confusion_matrix(labels=y_test2, predictions=pred2).numpy()

print(np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2))

[[0.62 0.38]
 [1.   0.  ]]


# Results

## Experimental Results
Fully Connected Model 1:  
Final RMSE score: 0.04718  
Accuracy: 0.9972799956040334  
Averaged F1: 0.997279914141294  
  
Fully Connected Model 2: 
Final RMSE score: 0.05672  
Accuracy: 0.9966206005989504  
Averaged F1: 0.9966201745337122  

Fully Connected Model 3: 
Final RMSE score: 0.04846  
Accuracy: 0.9970052476852488  
Averaged F1: 0.9970054088047998  

Fully Connected Model 4: 
Final RMSE score: 0.05589  
Accuracy: 0.996290903096409  
Averaged F1: 0.9962917640882064  

Fully Connected Model 5: 
Final RMSE score: 0.108  
Accuracy: 0.9862351292688958  
Averaged F1: 0.9862152862810428  

--

Convolutional Neural Network 1:  
Final RMSE score: 0.0579  
Accuracy: 0.9966480753908289  
Averaged F1: 0.996648115505803  

Convolutional Neural Network 2:  
Final RMSE score: 0.05572  
Accuracy: 0.996895348517735  
Averaged F1: 0.9968948822671937  

Convolutional Neural Network 3:  
Final RMSE score: 0.1272  
Accuracy: 0.9838173475835921  
Averaged F1: 0.9837850228009662  

Convolutional Neural Network 4:  
Final RMSE score: 0.1604  
Accuracy: 0.9742835948017694  
Averaged F1: 0.9741376672276821  

Convolutional Neural Network 5:  
Final RMSE score: 0.08708  
Accuracy: 0.9924169574415473  
Averaged F1: 0.9924159557228742  

## Summary
Best 3 models by RMSE:  

Fully Connected Model 1:  
Final RMSE score: 0.04718  
Accuracy: 0.9972799956040334  
Averaged F1: 0.997279914141294  
Layers: 50, 25, 10  
Activation: Relu  
Optimizer: Adam  

Fully Connected Model 3:  
Final RMSE score: 0.04846  
Accuracy: 0.9970052476852488  
Averaged F1: 0.9970054088047998  
Layers: 50, 25, 25, 25, 25  
Activation: Relu  
Optimizer: Adam  

Convolutional Neural Network 2:  
Final RMSE score: 0.05572  
Accuracy: 0.996895348517735  
Averaged F1: 0.9968948822671937  
Information See #CNN-2